# **Imports & Iperparameters**

In [ ]:
import tf_keras
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

import main.utilities.utils as us
import main.models.IV_RDN.architectures.architectures_rdn as arcs

from tf_keras.layers import *
from tf_keras.optimizers import Adam
from tf_keras.losses import MeanAbsoluteError

**Iperparametri** \
Questi sono gli Iperparametri che possiamo decidere noi

In [ ]:
# SuperRes
BATCH_SIZE   = 16
SCALE        = 2

# DataSet
LABEL_SIZE  = 64

# Model
TRAINING_STEPS = 50000
STEPSxEPOCHS = 1000
EPOCHS = TRAINING_STEPS // STEPSxEPOCHS


# Random Seed
AUTOTUNE = tf.data.AUTOTUNE
SEED        = 42
tf.random.set_seed(SEED)

# **Load Data**

In [ ]:
# Download DIV2K from TF Datasets
# Using bicubic 4x degradation type
div2k_data = tfds.image.Div2k(config="bicubic_x2")
div2k_data.download_and_prepare()

# Taking train data from div2k_data object
train = div2k_data.as_dataset(split="train", as_supervised=True)
# Validation data
val = div2k_data.as_dataset(split="validation", as_supervised=True)

train_cache = train.cache()
val_cache = val.cache()

**Visualize Training Data**

In [ ]:
samples = train.take(2)
ls = []
for sample in samples:
  lr = sample[0]
  hr = sample[1]
  ls.append(lr)
  ls.append(hr)
us.plot_images(ls)

# **Create Dataset & Preprocess Data**

In [ ]:
def dataset_object(dataset_cache, training=True):
  ds = dataset_cache

  # Random Crops
  ds = ds.map(
      lambda lowres, highres: us.random_crop(lowres, highres, hr_crop_size=LABEL_SIZE, scale=SCALE),
      num_parallel_calls=AUTOTUNE,
  )

  # Augment Data
  if training:
    ds = ds.map(us.random_rotate, num_parallel_calls=AUTOTUNE)
    ds = ds.map(us.random_flip, num_parallel_calls=AUTOTUNE)
    ds = ds.map(us.random_lr_jpeg_noise, num_parallel_calls=AUTOTUNE)

  # Batching Data
  ds = ds.batch(BATCH_SIZE)

  # Repeating Data, so that cardinality if dataset becomes infinte
  if training:
      ds = ds.repeat()

  # prefetching allows later images to be prepared while the current image is being processed
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  return ds

train_ds = dataset_object(train_cache, training=True)
val_ds = dataset_object(val_cache, training=False)

**Visualize the Data**

In [ ]:
lowres, highres = next(iter(train_ds))

# High Resolution Images
plt.figure(figsize=(10, 10))
for i in range(3):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(highres[i].numpy().astype("uint8"))
    plt.title(highres[i].shape)
    plt.axis("off")

# Low Resolution Images
plt.figure(figsize=(10, 10))
for i in range(3):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(lowres[i].numpy().astype("uint8"))
    plt.title(lowres[i].shape)
    plt.axis("off")


# **Model**

**1. Create the Model**

In [ ]:
model = arcs.RDN_Vanilla()
model._name = ("RDN_Vanilla_fit50_lr" + str(lr))

**2. Compile the Model**

In [ ]:
my_loss = MeanAbsoluteError();
my_opt = Adam(learning_rate=lr);

model.compile(
    loss=my_loss,
    optimizer=my_opt,
    metrics=[us.PSNR_metric, us.SSIM_metric]
)

In [ ]:
# Eseguiamo il Plot del Modello
model.summary()
us.plot_model(model, show_shapes=True, rankdir="LR")

**3. Fit the Model**

In [ ]:
# Define TensorBoard Callbacks
tb_callback = tf_keras.callbacks.TensorBoard(log_dir=("logs/" + str(lr)), histogram_freq=1)

In [ ]:
# Fit
model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, steps_per_epoch=STEPSxEPOCHS, callbacks=[tb_callback])

# **References**

[1] RDN-for-SR-by-keras - "https://github.com/cjdsj/RDN-for-SR-by-keras/blob/main/model.py"